In [15]:
from astroquery.heasarc import Heasarc

heasarc = Heasarc()

# ADQL query to get column names and metadata from fermigbrst table
adql_schema_query = """
SELECT * 
FROM TAP_SCHEMA.columns 
WHERE table_name='fermigbrst'
"""

print("Querying HEASARC TAP for fermigbrst table schema...")
schema_table = heasarc.query_tap(adql_schema_query)
print("Schema query complete.")

# Convert TAPResults to Astropy Table, then to pandas DataFrame for inspection
schema_df = schema_table.to_table().to_pandas()
print(schema_df[['column_name', 'datatype', 'description']])


Querying HEASARC TAP for fermigbrst table schema...
Schema query complete.
                 column_name datatype  \
0    flnc_band_phtflnc_error   double   
1          flnc_sbpl_ergflux   double   
2         flnc_plaw_phtfluxb   double   
3          flnc_band_ergflux   double   
4          flnc_sbpl_phtflux   double   
..                       ...      ...   
305                  fluence   double   
306       pflx_comp_redchisq   double   
307   back_interval_low_stop   double   
308  flnc_sbpl_brken_neg_err   double   
309  pflx_band_ergflnc_error   double   

                                           description  
0    Band Model Photon Fluence Error for Fluence Sp...  
1                SBPL Energy Flux for Fluence Spectrum  
2    Power Law Model 50-300 keV (BATSE) Photon Flux...  
3          Band Model Energy Flux for Fluence Spectrum  
4                SBPL Photon Flux for Fluence Spectrum  
..                                                 ...  
305                              

In [16]:
schema_df.to_csv('schema_df.csv')

In [ ]:
from astroquery.heasarc import Heasarc
from astropy.time import Time
import pandas as pd

heasarc = Heasarc()

start_date = '2017-07-01'
end_date = '2017-09-30'

# convert to MJD numbers
start_mjd = Time(start_date).mjd
end_mjd = Time(end_date).mjd

adql_data_query = f"""
SELECT trigger_name, trigger_time, ra, dec, error_radius, flux_256
FROM fermigbrst
WHERE trigger_time BETWEEN {start_mjd} AND {end_mjd}
"""

print("Querying HEASARC for Fermi GRB data in August 2017 with correct MJD range...")
grb_table = heasarc.query_tap(adql_data_query)
print("Data query complete.")

raw_heasarc_df = grb_table.to_table().to_pandas()
print(f"Downloaded {len(raw_heasarc_df)} events.")

column_mapping = {
    'trigger_name': 'event_id',
    'trigger_time': 'utc_time',
    'ra': 'ra_deg',
    'dec': 'dec_deg',
    'error_radius': 'pos_error_deg',
    'flux_256': 'signal_strength'
}

heasarc_df = raw_heasarc_df.rename(columns=column_mapping)

# Convert MJD numeric trigger_time to datetime using astropy
heasarc_df['utc_time'] = Time(heasarc_df['utc_time'], format='mjd').to_datetime()

# Convert other numeric columns
for col in ['ra_deg', 'dec_deg', 'pos_error_deg', 'signal_strength']:
    heasarc_df[col] = pd.to_numeric(heasarc_df[col], errors='coerce')

heasarc_df.dropna(inplace=True)

# Add metadata columns
heasarc_df['source'] = 'HEASARC'
heasarc_df['event_type'] = 'GRB'

print("\n--- Standardized HEASARC DataFrame ---")
print(heasarc_df.head())
print("\nData types of the final columns:")
print(heasarc_df.info())




Querying HEASARC for Fermi GRB data in August 2017 with correct MJD range...
Data query complete.
Downloaded 251 events.

--- Standardized HEASARC DataFrame ---
      event_id                   utc_time  ra_deg  dec_deg  pos_error_deg  \
0  bn170116238 2017-01-16 05:43:15.259239   72.94   -87.43           9.37   
1  bn170130697 2017-01-30 16:43:13.295769  296.39   -80.45          10.73   
2  bn171206122 2017-12-06 02:55:44.936745    9.48   -78.20           6.03   
3  bn170124874 2017-01-24 20:58:06.357821  282.04   -75.51           1.00   
4  bn170121067 2017-01-21 01:36:53.640605    3.03   -75.62           4.07   

   signal_strength   source event_type  
0          6.49597  HEASARC        GRB  
1          2.20144  HEASARC        GRB  
2          6.62104  HEASARC        GRB  
3          6.36266  HEASARC        GRB  
4          6.79250  HEASARC        GRB  

Data types of the final columns:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 8 co

In [27]:
heasarc_df.to_csv('heasarc_df_year_2017.csv')

In [28]:
import pandas as pd
import numpy as np
from astropy.time import Time
from astropy.coordinates import SkyCoord
from astroquery.simbad import Simbad
import astropy.units as u

# --- STEP 1: LOAD AND PREPARE THE DATASET ---

# Load the CSV file you provided
try:
    master_df = pd.read_csv('heasarc_df.csv')
    print("Successfully loaded heasarc_df.csv")
except FileNotFoundError:
    print("Error: heasarc_df.csv not found. Please ensure the file is in the correct directory.")
    exit()

# Ensure the utc_time column is in datetime format
master_df['utc_time'] = pd.to_datetime(master_df['utc_time'])

# Sort the master list by time, which is crucial for the search algorithm
master_df.sort_values(by='utc_time', inplace=True)
master_df.reset_index(drop=True, inplace=True)

# Feature Engineering: Calculate percentile rank for signal_strength
# Since we only have one event type (GRB), this is straightforward
master_df['signal_percentile'] = master_df['signal_strength'].rank(pct=True)

print(f"Loaded and prepared {len(master_df)} events for analysis.")


# --- STEP 2: DEFINE THE CORE ENGINE LOGIC ---

# [cite_start]Configuration Engine [cite: 1]
EVENT_CONFIG = {
    'GRB': {'time_window_days': 0.5, 'max_search_radius_deg': 5.0},
    'DEFAULT': {'time_window_days': 1.0, 'max_search_radius_deg': 10.0}
}

# Scoring Weights
WEIGHTS = {'W_st': 0.5, 'W_sig': 0.3, 'W_ctx': 0.2}

# Helper function to calculate angular distance
def calculate_angular_distance(ra1, dec1, ra2, dec2):
    c1 = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree, frame='icrs')
    c2 = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree, frame='icrs')
    return c1.separation(c2).degree

# The main scoring function
def calculate_scores(event_A, event_B):
    features = {}
    
    # [cite_start]Feature Engineering based on your proposal's Phase 3 [cite: 1]
    features['time_delta_sec'] = abs((event_A['utc_time'] - event_B['utc_time']).total_seconds())
    features['angular_dist_deg'] = calculate_angular_distance(event_A['ra_deg'], event_A['dec_deg'], event_B['ra_deg'], event_B['dec_deg'])
    
    # [cite_start]This is your 'significance_of_match' feature [cite: 1]
    # Adding a small epsilon to avoid division by zero
    features['S_match'] = features['angular_dist_deg'] / (event_A['pos_error_deg'] + event_B['pos_error_deg'] + 1e-6)
    
    # --- Calculate Sub-scores ---
    # 1. Spatio-Temporal Score
    score_spatial = np.exp(-features['S_match'])
    score_temporal = np.exp(-features['time_delta_sec'] / 3600) # Characteristic timescale of 1 hour
    features['score_spatiotemporal'] = score_spatial * score_temporal

    # 2. Significance Score
    norm_sig_A = event_A['signal_percentile']
    norm_sig_B = event_B['signal_percentile']
    features['score_significance'] = norm_sig_A * norm_sig_B

    # 3. Contextual Score
    try:
        # Check for a known galaxy near the event's location
        result_table = Simbad.query_region(SkyCoord(ra=event_A['ra_deg']*u.degree, dec=event_A['dec_deg']*u.degree), radius=0.5*u.degree)
        is_galaxy_nearby = any('G' in t for t in result_table['OTYPE']) if result_table else False
        features['score_contextual'] = 1.0 if is_galaxy_nearby else 0.1
    except Exception:
        features['score_contextual'] = 0.1 # Default to low score on error if query fails

    # --- Final Confidence Score ---
    features['confidence_score'] = (WEIGHTS['W_st'] * features['score_spatiotemporal'] +
                                    WEIGHTS['W_sig'] * features['score_significance'] +
                                    WEIGHTS['W_ctx'] * features['score_contextual'])
    
    return features


# --- STEP 3: RUN THE ADAPTIVE SEARCH AND SCORE CANDIDATES ---

scored_candidates = []
print("\n--- Starting Adaptive Candidate Search ---")

for i in range(len(master_df)):
    event_A = master_df.iloc[i]
    config = EVENT_CONFIG.get(event_A['event_type'], EVENT_CONFIG['DEFAULT'])
    
    # Inner loop for subsequent events (time-windowed search)
    for j in range(i + 1, len(master_df)):
        event_B = master_df.iloc[j]
        
        # Time Check: If the next event is outside the time window, break the inner loop
        time_delta = event_B['utc_time'] - event_A['utc_time']
        if time_delta.days > config['time_window_days']:
            break
            
        # Spatial Check
        angular_dist = calculate_angular_distance(event_A['ra_deg'], event_A['dec_deg'], event_B['ra_deg'], event_B['dec_deg'])
        if angular_dist < config['max_search_radius_deg']:
            # This is a candidate pair, now score it
            scores = calculate_scores(event_A, event_B)
            
            result = {
                'event_A_id': event_A['event_id'],
                'event_B_id': event_B['event_id'],
                'time_delta_sec': scores['time_delta_sec'],
                'angular_dist_deg': scores['angular_dist_deg'],
                'confidence_score': scores['confidence_score'],
                'score_spatiotemporal': scores['score_spatiotemporal'],
                'score_significance': scores['score_significance'],
                'score_contextual': scores['score_contextual']
            }
            scored_candidates.append(result)

print(f"Search complete. Found {len(scored_candidates)} candidate pairs.")

# --- STEP 4: FINAL RANKING AND OUTPUT ---

if scored_candidates:
    # Convert the list of results into a final DataFrame
    final_results_df = pd.DataFrame(scored_candidates)

    # Sort by the confidence score to find the best candidates
    final_results_df.sort_values(by='confidence_score', ascending=False, inplace=True)
    final_results_df.reset_index(drop=True, inplace=True)

    print("\n--- Top 10 Correlated Event Candidates ---")
    print(final_results_df.head(10).to_string())
else:
    print("\nNo correlated event candidates found within the specified parameters.")

Successfully loaded heasarc_df.csv
Loaded and prepared 28 events for analysis.

--- Starting Adaptive Candidate Search ---
Search complete. Found 0 candidate pairs.

No correlated event candidates found within the specified parameters.


In [3]:
import pandas as pd
import numpy as np
from astropy.coordinates import SkyCoord
from astroquery.simbad import Simbad
import astropy.units as u

# --- STEP 1: LOAD AND COMBINE YOUR DATASETS ---

# --- STEP 1: LOAD AND COMBINE YOUR DATASETS (Corrected for Timezone) ---

try:
    # Load the HEASARC GRB data
    heasarc_df = pd.read_csv('heasarc_df_year_2017.csv')
    print(f"Successfully loaded {len(heasarc_df)} events from heasarc_df_year_2017.csv")
    
    # Load the GWOSC GW data
    gw_df = pd.read_csv('gw_events_complete3.csv')
    print(f"Successfully loaded {len(gw_df)} events from gw_events_complete3.csv")

except FileNotFoundError as e:
    print(f"Error: Could not find a data file. Make sure both CSV files are in the correct directory.")
    print(e)
    exit()

# Standardize the utc_time column to datetime objects
heasarc_df['utc_time'] = pd.to_datetime(heasarc_df['utc_time'])
gw_df['utc_time'] = pd.to_datetime(gw_df['utc_time'])

# --- FIX: Ensure both columns are timezone-aware and set to UTC ---
heasarc_df['utc_time'] = heasarc_df['utc_time'].dt.tz_localize(None).dt.tz_localize('UTC')
gw_df['utc_time'] = gw_df['utc_time'].dt.tz_localize(None).dt.tz_localize('UTC')

# Combine into a single master DataFrame
master_df = pd.concat([heasarc_df, gw_df], ignore_index=True)

# Sort the master list by time, which is crucial for the search algorithm
master_df.sort_values(by='utc_time', inplace=True)
master_df.reset_index(drop=True, inplace=True)

# Feature Engineering: Calculate percentile rank for signal_strength within each event type
master_df['signal_percentile'] = master_df.groupby('event_type')['signal_strength'].rank(pct=True)

print(f"\nCombined and prepared a total of {len(master_df)} events for analysis.")


# --- STEP 2: DEFINE THE CORE ENGINE LOGIC ---

# Configuration Engine for the adaptive search
EVENT_CONFIG = {
    'GW': {'time_window_days': 2.0},
    'GRB': {'time_window_days': 0.5},
    'DEFAULT': {'time_window_days': 1.0}
}

# Scoring Weights for the final confidence score
WEIGHTS = {'W_st': 0.5, 'W_sig': 0.3, 'W_ctx': 0.2}

# Helper function to calculate angular distance between two points on the sky
def calculate_angular_distance(ra1, dec1, ra2, dec2):
    c1 = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree, frame='icrs')
    c2 = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree, frame='icrs')
    return c1.separation(c2).degree

# The main scoring function that calculates all features and scores for a pair
def calculate_scores(event_A, event_B):
    features = {}
    
    # Feature Engineering
    features['time_delta_sec'] = abs((event_A['utc_time'] - event_B['utc_time']).total_seconds())
    features['angular_dist_deg'] = calculate_angular_distance(event_A['ra_deg'], event_A['dec_deg'], event_B['ra_deg'], event_B['dec_deg'])
    features['S_match'] = features['angular_dist_deg'] / (event_A['pos_error_deg'] + event_B['pos_error_deg'] + 1e-6)
    
    # --- Calculate Sub-scores ---
    # 1. Spatio-Temporal Score
    score_spatial = np.exp(-features['S_match'])
    score_temporal = np.exp(-features['time_delta_sec'] / 3600) # Characteristic timescale of 1 hour
    features['score_spatiotemporal'] = score_spatial * score_temporal

    # 2. Significance Score
    norm_sig_A = event_A['signal_percentile']
    norm_sig_B = event_B['signal_percentile']
    features['score_significance'] = norm_sig_A * norm_sig_B

    # 3. Contextual Score (with error handling)
    try:
        result_table = Simbad.query_region(SkyCoord(ra=event_A['ra_deg']*u.degree, dec=event_A['dec_deg']*u.degree), radius=0.5*u.degree)
        is_galaxy_nearby = any('G' in t for t in result_table['OTYPE']) if result_table else False
        features['score_contextual'] = 1.0 if is_galaxy_nearby else 0.1
    except Exception:
        features['score_contextual'] = 0.1 # Default to low score on query error

    # --- Final Confidence Score ---
    features['confidence_score'] = (WEIGHTS['W_st'] * features['score_spatiotemporal'] +
                                    WEIGHTS['W_sig'] * features['score_significance'] +
                                    WEIGHTS['W_ctx'] * features['score_contextual'])
    
    return features


# --- STEP 3: RUN THE ADAPTIVE SEARCH AND SCORE CANDIDATES ---

scored_candidates = []
print("\n--- Starting Adaptive Candidate Search ---")

for i in range(len(master_df)):
    event_A = master_df.iloc[i]
    config = EVENT_CONFIG.get(event_A['event_type'], EVENT_CONFIG['DEFAULT'])
    
    for j in range(i + 1, len(master_df)):
        event_B = master_df.iloc[j]
        
        # Time Check: Break inner loop if outside the adaptive time window
        time_delta = event_B['utc_time'] - event_A['utc_time']
        if time_delta.days > config['time_window_days']:
            break
            
        # Spatial Check: Only consider events where the angular distance is less than the SUM of their error radii
        # This is a more intelligent check than a fixed radius.
        if calculate_angular_distance(event_A['ra_deg'], event_A['dec_deg'], event_B['ra_deg'], event_B['dec_deg']) < (event_A['pos_error_deg'] + event_B['pos_error_deg']):
            scores = calculate_scores(event_A, event_B)
            
            result = {
                'event_A_id': event_A['event_id'],
                'event_A_type': event_A['event_type'],
                'event_B_id': event_B['event_id'],
                'event_B_type': event_B['event_type'],
                'confidence_score': scores['confidence_score'],
                'score_spatiotemporal': scores['score_spatiotemporal'],
                'score_significance': scores['score_significance'],
                'score_contextual': scores['score_contextual'],
                'time_delta_sec': scores['time_delta_sec'],
                'angular_dist_deg': scores['angular_dist_deg'],
            }
            scored_candidates.append(result)

print(f"Search complete. Found {len(scored_candidates)} candidate pairs.")

# --- STEP 4: FINAL RANKING AND OUTPUT ---

if scored_candidates:
    final_results_df = pd.DataFrame(scored_candidates)
    final_results_df.sort_values(by='confidence_score', ascending=False, inplace=True)
    final_results_df.reset_index(drop=True, inplace=True)

    print("\n--- Top 10 Correlated Event Candidates ---")
    # Displaying with formatted columns for better readability
    print(final_results_df.head(10).to_string(formatters={
        'confidence_score': '{:,.3f}'.format,
        'score_spatiotemporal': '{:,.3f}'.format,
        'score_significance': '{:,.3f}'.format,
        'score_contextual': '{:,.1f}'.format,
        'time_delta_sec': '{:,.1f}'.format,
        'angular_dist_deg': '{:,.3f}'.format
    }))
else:
    print("\nNo correlated event candidates found within the specified parameters.")

Successfully loaded 251 events from heasarc_df_year_2017.csv
Successfully loaded 47 events from gw_events_complete3.csv

Combined and prepared a total of 298 events for analysis.

--- Starting Adaptive Candidate Search ---
Search complete. Found 13 candidate pairs.

--- Top 10 Correlated Event Candidates ---
        event_A_id event_A_type       event_B_id event_B_type confidence_score score_spatiotemporal score_significance score_contextual time_delta_sec angular_dist_deg
0      bn170817529          GRB         GW170817           GW            0.572                0.990              0.191              0.1           34.9            0.001
1         GW190412           GW  GW190413_052954           GW            0.126                0.000              0.353              0.1       86,350.4           20.928
2  GW190706_222641           GW  GW190707_093326           GW            0.068                0.000              0.159              0.1       40,004.8           37.954
3  GW190803_022701

correct script

In [2]:
import pandas as pd
import numpy as np
from astropy.coordinates import SkyCoord
from astroquery.simbad import Simbad
import astropy.units as u

# --- STEP 1: LOAD AND COMBINE YOUR DATASETS (WITH TIMEZONE FIX) ---

try:
    heasarc_df = pd.read_csv('heasarc_df_year_2017.csv')
    print(f"Successfully loaded {len(heasarc_df)} events from heasarc_df_year_2017.csv")
    gw_df = pd.read_csv('gw_events_complete3.csv')
    print(f"Successfully loaded {len(gw_df)} events from gw_events_complete3.csv")
except FileNotFoundError as e:
    print(f"Error: Could not find a data file. Make sure both CSV files are in the correct directory.")
    print(e)
    exit()

# Convert to datetime objects first
heasarc_df['utc_time'] = pd.to_datetime(heasarc_df['utc_time'])
gw_df['utc_time'] = pd.to_datetime(gw_df['utc_time'])

# ### FIX 1: TIMEZONE CORRECTION ###
# Standardize both DataFrames to the UTC timezone to ensure correct time difference calculations.
heasarc_df['utc_time'] = heasarc_df['utc_time'].dt.tz_localize('UTC')
gw_df['utc_time'] = gw_df['utc_time'].dt.tz_localize('UTC')

master_df = pd.concat([heasarc_df, gw_df], ignore_index=True)
master_df.sort_values(by='utc_time', inplace=True)
master_df.reset_index(drop=True, inplace=True)

# ### FIX 3: IMPROVED SIGNIFICANCE NORMALIZATION ###
# Use a log-scale followed by min-max scaling to better represent the order-of-magnitude
# importance of signal strength, instead of a simple percentile rank.
master_df['log_signal'] = np.log10(master_df['signal_strength'] + 1) # Add 1 to avoid log(0)
master_df['signal_norm_score'] = master_df.groupby('event_type')['log_signal'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min()) if (x.max() - x.min()) > 0 else 0
)

print(f"\nCombined and prepared a total of {len(master_df)} events for analysis.")


# --- STEP 2: DEFINE THE CORE ENGINE LOGIC (WITH CONTEXTUAL FIX) ---

EVENT_CONFIG = {
    'GW': {'time_window_days': 2.0},
    'GRB': {'time_window_days': 0.5},
    'DEFAULT': {'time_window_days': 1.0}
}
WEIGHTS = {'W_st': 0.5, 'W_sig': 0.3, 'W_ctx': 0.2}

def calculate_angular_distance(ra1, dec1, ra2, dec2):
    c1 = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree, frame='icrs')
    c2 = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree, frame='icrs')
    return c1.separation(c2).degree

def calculate_scores(event_A, event_B):
    features = {}
    features['time_delta_sec'] = abs((event_A['utc_time'] - event_B['utc_time']).total_seconds())
    features['angular_dist_deg'] = calculate_angular_distance(event_A['ra_deg'], event_A['dec_deg'], event_B['ra_deg'], event_B['dec_deg'])
    features['S_match'] = features['angular_dist_deg'] / (event_A['pos_error_deg'] + event_B['pos_error_deg'] + 1e-6)
    
    score_spatial = np.exp(-features['S_match'])
    score_temporal = np.exp(-features['time_delta_sec'] / 3600)
    features['score_spatiotemporal'] = score_spatial * score_temporal

    # Use the improved normalized score
    features['score_significance'] = event_A['signal_norm_score'] * event_B['signal_norm_score']

    # ### FIX 2: ROBUST CONTEXTUAL SCORE ###
    # Make the SIMBAD query more robust and improve the check for galaxy types.
    try:
        # Query a slightly larger region to ensure we find the host
        search_radius = max(0.5, event_A['pos_error_deg']) * u.degree
        result_table = Simbad.query_region(SkyCoord(ra=event_A['ra_deg']*u.degree, dec=event_A['dec_deg']*u.degree), radius=search_radius)
        
        # Check for various galaxy classifications, not just 'G'
        galaxy_types = ['G', 'Galaxy', 'LINER', 'Seyfert', 'PartofG']
        is_galaxy_nearby = any(any(gt in t for gt in galaxy_types) for t in result_table['OTYPE']) if result_table else False
        features['score_contextual'] = 1.0 if is_galaxy_nearby else 0.1
    except Exception as e:
        # print(f"SIMBAD query failed for {event_A['event_id']}: {e}") # Uncomment for debugging
        features['score_contextual'] = 0.1

    features['confidence_score'] = (WEIGHTS['W_st'] * features['score_spatiotemporal'] +
                                    WEIGHTS['W_sig'] * features['score_significance'] +
                                    WEIGHTS['W_ctx'] * features['score_contextual'])
    
    return features


# --- STEP 3: RUN THE ADAPTIVE SEARCH AND SCORE CANDIDATES (Unchanged) ---
# The logic here remains the same, but it will now use the corrected functions and data.
scored_candidates = []
print("\n--- Starting Adaptive Candidate Search with Corrected Logic ---")
# ... (The rest of the script from the previous response remains the same) ...
for i in range(len(master_df)):
    event_A = master_df.iloc[i]
    config = EVENT_CONFIG.get(event_A['event_type'], EVENT_CONFIG['DEFAULT'])
    
    for j in range(i + 1, len(master_df)):
        event_B = master_df.iloc[j]
        
        time_delta = event_B['utc_time'] - event_A['utc_time']
        if time_delta.days > config['time_window_days']:
            break
            
        if calculate_angular_distance(event_A['ra_deg'], event_A['dec_deg'], event_B['ra_deg'], event_B['dec_deg']) < (event_A['pos_error_deg'] + event_B['pos_error_deg']):
            scores = calculate_scores(event_A, event_B)
            
            result = {
                'event_A_id': event_A['event_id'], 'event_A_type': event_A['event_type'],
                'event_B_id': event_B['event_id'], 'event_B_type': event_B['event_type'],
                'confidence_score': scores['confidence_score'], 'score_spatiotemporal': scores['score_spatiotemporal'],
                'score_significance': scores['score_significance'], 'score_contextual': scores['score_contextual'],
                'time_delta_sec': scores['time_delta_sec'], 'angular_dist_deg': scores['angular_dist_deg'],
            }
            scored_candidates.append(result)

print(f"Search complete. Found {len(scored_candidates)} candidate pairs.")

# --- STEP 4: FINAL RANKING AND OUTPUT ---
if scored_candidates:
    final_results_df = pd.DataFrame(scored_candidates)
    final_results_df.sort_values(by='confidence_score', ascending=False, inplace=True)
    final_results_df.reset_index(drop=True, inplace=True)
    print("\n--- Top 10 Correlated Event Candidates (Corrected) ---")
    print(final_results_df.head(10).to_string(formatters={
        'confidence_score': '{:,.3f}'.format, 'score_spatiotemporal': '{:,.3f}'.format,
        'score_significance': '{:,.3f}'.format, 'score_contextual': '{:,.1f}'.format,
        'time_delta_sec': '{:,.1f}'.format, 'angular_dist_deg': '{:,.3f}'.format
    }))
else:
    print("\nNo correlated event candidates found within the specified parameters.")

Successfully loaded 251 events from heasarc_df_year_2017.csv
Successfully loaded 47 events from gw_events_complete3.csv

Combined and prepared a total of 298 events for analysis.

--- Starting Adaptive Candidate Search with Corrected Logic ---


KeyboardInterrupt: 

In [1]:
import pandas as pd
import numpy as np
from astropy.coordinates import SkyCoord
from astroquery.simbad import Simbad
import astropy.units as u
from tqdm import tqdm # Import the tqdm library

# --- STEP 1: LOAD AND COMBINE YOUR DATASETS (Corrected for Timezone) ---
# (This part remains the same)
try:
    heasarc_df = pd.read_csv('heasarc_df_year_2017.csv')
    gw_df = pd.read_csv('gw_events_complete3.csv')
except FileNotFoundError as e:
    print(f"Error: Could not find a data file.")
    exit()

heasarc_df['utc_time'] = pd.to_datetime(heasarc_df['utc_time']).dt.tz_localize('UTC')
gw_df['utc_time'] = pd.to_datetime(gw_df['utc_time']).dt.tz_localize('UTC')

master_df = pd.concat([heasarc_df, gw_df], ignore_index=True)
master_df.sort_values(by='utc_time', inplace=True)
master_df.reset_index(drop=True, inplace=True)

master_df['log_signal'] = np.log10(master_df['signal_strength'] + 1)
master_df['signal_norm_score'] = master_df.groupby('event_type')['log_signal'].transform(
    lambda x: (x - x.min()) / (x.max() - x.min()) if (x.max() - x.min()) > 0 else 0
)

print(f"Combined and prepared a total of {len(master_df)} events for analysis.")


# --- STEP 2: DEFINE THE CORE ENGINE LOGIC (Unchanged) ---
# (The configuration and functions remain the same)
EVENT_CONFIG = {
    'GW': {'time_window_days': 2.0},
    'GRB': {'time_window_days': 0.5},
    'DEFAULT': {'time_window_days': 1.0}
}
WEIGHTS = {'W_st': 0.5, 'W_sig': 0.3, 'W_ctx': 0.2}

def calculate_angular_distance(ra1, dec1, ra2, dec2):
    c1 = SkyCoord(ra=ra1*u.degree, dec=dec1*u.degree, frame='icrs')
    c2 = SkyCoord(ra=ra2*u.degree, dec=dec2*u.degree, frame='icrs')
    return c1.separation(c2).degree

def calculate_scores(event_A, event_B):
    features = {}
    features['time_delta_sec'] = abs((event_A['utc_time'] - event_B['utc_time']).total_seconds())
    features['angular_dist_deg'] = calculate_angular_distance(event_A['ra_deg'], event_A['dec_deg'], event_B['ra_deg'], event_B['dec_deg'])
    features['S_match'] = features['angular_dist_deg'] / (event_A['pos_error_deg'] + event_B['pos_error_deg'] + 1e-6)
    
    score_spatial = np.exp(-features['S_match'])
    score_temporal = np.exp(-features['time_delta_sec'] / 3600)
    features['score_spatiotemporal'] = score_spatial * score_temporal

    features['score_significance'] = event_A['signal_norm_score'] * event_B['signal_norm_score']

    try:
        search_radius = max(0.5, event_A['pos_error_deg']) * u.degree
        result_table = Simbad.query_region(SkyCoord(ra=event_A['ra_deg']*u.degree, dec=event_A['dec_deg']*u.degree), radius=search_radius)
        galaxy_types = ['G', 'Galaxy', 'LINER', 'Seyfert', 'PartofG']
        is_galaxy_nearby = any(any(gt in t for gt in galaxy_types) for t in result_table['OTYPE']) if result_table else False
        features['score_contextual'] = 1.0 if is_galaxy_nearby else 0.1
    except Exception as e:
        features['score_contextual'] = 0.1

    features['confidence_score'] = (WEIGHTS['W_st'] * features['score_spatiotemporal'] +
                                    WEIGHTS['W_sig'] * features['score_significance'] +
                                    WEIGHTS['W_ctx'] * features['score_contextual'])
    return features


# --- STEP 3: RUN THE ADAPTIVE SEARCH WITH PROGRESS BAR ---

scored_candidates = []
print("\n--- Starting Adaptive Candidate Search ---")

# ### PROGRESS BAR INTEGRATION ###
# Wrap the main loop's iterator with tqdm() to create the progress bar.
for i in tqdm(range(len(master_df)), desc="Processing Events"):
    event_A = master_df.iloc[i]
    config = EVENT_CONFIG.get(event_A['event_type'], EVENT_CONFIG['DEFAULT'])
    
    for j in range(i + 1, len(master_df)):
        event_B = master_df.iloc[j]
        
        time_delta = event_B['utc_time'] - event_A['utc_time']
        if time_delta.days > config['time_window_days']:
            break
            
        if calculate_angular_distance(event_A['ra_deg'], event_A['dec_deg'], event_B['ra_deg'], event_B['dec_deg']) < (event_A['pos_error_deg'] + event_B['pos_error_deg']):
            scores = calculate_scores(event_A, event_B)
            
            result = {
                'event_A_id': event_A['event_id'], 'event_A_type': event_A['event_type'],
                'event_B_id': event_B['event_id'], 'event_B_type': event_B['event_type'],
                'confidence_score': scores['confidence_score'], 'score_spatiotemporal': scores['score_spatiotemporal'],
                'score_significance': scores['score_significance'], 'score_contextual': scores['score_contextual'],
                'time_delta_sec': scores['time_delta_sec'], 'angular_dist_deg': scores['angular_dist_deg'],
            }
            scored_candidates.append(result)

print(f"\nSearch complete. Found {len(scored_candidates)} candidate pairs.")


# --- STEP 4: FINAL RANKING AND OUTPUT (Unchanged) ---
# (This part remains the same)
if scored_candidates:
    final_results_df = pd.DataFrame(scored_candidates)
    final_results_df.sort_values(by='confidence_score', ascending=False, inplace=True)
    final_results_df.reset_index(drop=True, inplace=True)
    print("\n--- Top 10 Correlated Event Candidates ---")
    print(final_results_df.head(10).to_string(formatters={
        'confidence_score': '{:,.3f}'.format, 'score_spatiotemporal': '{:,.3f}'.format,
        'score_significance': '{:,.3f}'.format, 'score_contextual': '{:,.1f}'.format,
        'time_delta_sec': '{:,.1f}'.format, 'angular_dist_deg': '{:,.3f}'.format
    }))
else:
    print("\nNo correlated event candidates found within the specified parameters.")

Combined and prepared a total of 298 events for analysis.

--- Starting Adaptive Candidate Search ---


Processing Events: 100%|██████████| 298/298 [1:20:39<00:00, 16.24s/it] 


Search complete. Found 13 candidate pairs.

--- Top 10 Correlated Event Candidates ---
        event_A_id event_A_type       event_B_id event_B_type confidence_score score_spatiotemporal score_significance score_contextual time_delta_sec angular_dist_deg
0      bn170817529          GRB         GW170817           GW            0.549                0.990              0.115              0.1           34.9            0.001
1         GW190412           GW  GW190413_052954           GW            0.041                0.000              0.068              0.1       86,350.4           20.928
2  GW190517_055101           GW  GW190519_153544           GW            0.029                0.000              0.031              0.1      207,882.6           49.552
3      bn170804911          GRB      bn170805901          GRB            0.029                0.000              0.029              0.1       85,528.1           14.275
4  GW190706_222641           GW  GW190707_093326           GW           

In [2]:
final_results_df.to_csv('final_results_df.csv')